# Day 10: Week 2 Review - Position & Trend Trading

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week2_position_trend/day10_week2_review.ipynb)

---

## Learning Objectives

1. **Review** all Week 2 concepts: trend, breakout, sector, earnings
2. **Integrate** multiple strategies into unified analysis
3. **Build** a comprehensive position trading scanner
4. **Create** a complete trade plan combining all methods
5. **Apply** Week 2 strategies to real market data

---

## Lecture (30 minutes)

### Week 2 Summary

```
WEEK 2: POSITION & TREND TRADING
================================

DAY 6: TREND FOLLOWING
- Moving average crossovers (Golden/Death Cross)
- ADX for trend strength (>25 = trending)
- Donchian Channels for breakouts
- Turtle Trading system principles

DAY 7: BREAKOUT STRATEGIES
- Horizontal and pattern breakouts
- Volume confirmation (>1.5x average)
- False breakout identification
- Stop placement (below breakout level)

DAY 8: SECTOR ROTATION
- Business cycle and sector performance
- Relative strength vs SPY
- Cyclical vs defensive sectors
- Rotation into strength

DAY 9: EARNINGS TRADING
- Pre-earnings momentum plays
- Post-earnings gap strategies
- Gap and go vs gap fade
- Risk management around earnings
```

### Strategy Comparison

```
STRATEGY COMPARISON
===================

                    HOLDING     TRADES/     CAPITAL     BEST
STRATEGY            PERIOD      MONTH       NEEDED      FOR
------------------------------------------------------------
Trend Following     Weeks-Mon   2-4         $25K+       Trending
Breakouts           Days-Weeks  4-8         $10K+       Ranging→Trend
Sector Rotation     Weeks-Mon   1-2         $50K+       All markets
Earnings Gaps       1-5 days    4-8         $10K+       Earnings season

WIN RATE / PROFIT FACTOR:
- Trend Following: 35-45% / 2.0-3.0x
- Breakouts: 40-50% / 1.5-2.0x
- Sector Rotation: 50-60% / 1.3-1.5x
- Earnings Gaps: 45-55% / 1.5-2.0x
```

### When to Use Each Strategy

```
STRATEGY SELECTION FLOWCHART
============================

                    MARKET CONDITIONS
                          |
    +---------------------+---------------------+
    |                     |                     |
TRENDING               RANGING              UNCERTAIN
    |                     |                     |
    v                     v                     v
Trend Following      Breakout Watch       Sector Rotation
                     (wait for break)     (defensive sectors)
    |
    +-- Earnings Coming? --> Earnings Strategy
    |                        (reduce size/exit)
    |
    +-- Strong Sector? --> Combine with
                          Sector Rotation
```

### Building a Complete System

```
INTEGRATED POSITION TRADING SYSTEM
==================================

STEP 1: MACRO ANALYSIS
[ ] Check market trend (SPY above 200 MA?)
[ ] Identify leading/lagging sectors
[ ] Note upcoming earnings dates

STEP 2: SECTOR SELECTION
[ ] Find sectors with rising RS vs SPY
[ ] Focus on top 2-3 sectors
[ ] Avoid weakest sectors

STEP 3: STOCK SELECTION
[ ] Screen for trending stocks in strong sectors
[ ] Look for breakout setups
[ ] Check earnings dates (avoid if within 2 weeks)

STEP 4: ENTRY TIMING
[ ] Wait for trend confirmation or breakout
[ ] Volume must confirm
[ ] Enter on pullback to MA or breakout retest

STEP 5: RISK MANAGEMENT
[ ] Position size based on stop distance
[ ] Stop below MA or breakout level
[ ] Target 2:1 or 3:1 R:R minimum
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
print("Libraries loaded!")

In [ ]:
def fetch_data(ticker, period='1y'):
    """Fetch historical data."""
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)
        df.index = pd.to_datetime(df.index)
        if df.index.tz is not None:
            df.index = df.index.tz_localize(None)
        return df
    except:
        return None

def fetch_multiple(tickers, period='1y'):
    """Fetch data for multiple tickers."""
    data = {}
    for ticker in tickers:
        df = fetch_data(ticker, period)
        if df is not None and len(df) > 50:
            data[ticker] = df
    return data

# Test stocks and sectors
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'JPM', 'XOM', 'UNH', 'HD']
sectors = {'XLK': 'Technology', 'XLF': 'Financials', 'XLE': 'Energy', 
           'XLV': 'Healthcare', 'XLY': 'Consumer Disc.', 'XLI': 'Industrials'}

print("Fetching data...")
stock_data = fetch_multiple(stocks)
sector_data = fetch_multiple(list(sectors.keys()) + ['SPY'])
print(f"Loaded {len(stock_data)} stocks and {len(sector_data)} sector ETFs")

In [ ]:
def add_all_indicators(df):
    """Add all Week 2 indicators to dataframe."""
    df = df.copy()
    
    # Trend indicators
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()
    df['SMA_200'] = df['Close'].rolling(200).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    
    # ADX for trend strength
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    plus_dm = high.diff()
    minus_dm = low.diff().abs() * -1
    plus_dm = plus_dm.where((plus_dm > minus_dm.abs()) & (plus_dm > 0), 0)
    minus_dm = minus_dm.abs().where((minus_dm.abs() > plus_dm) & (minus_dm < 0), 0)
    
    tr = pd.concat([high - low, (high - close.shift()).abs(), (low - close.shift()).abs()], axis=1).max(axis=1)
    atr = tr.rolling(14).mean()
    
    plus_di = 100 * (plus_dm.rolling(14).mean() / atr)
    minus_di = 100 * (minus_dm.rolling(14).mean() / atr)
    dx = 100 * ((plus_di - minus_di).abs() / (plus_di + minus_di))
    df['ADX'] = dx.rolling(14).mean()
    
    # Breakout levels
    df['Resistance_20'] = df['High'].rolling(20).max().shift(1)
    df['Support_20'] = df['Low'].rolling(20).min().shift(1)
    
    # Volume
    df['Vol_Avg'] = df['Volume'].rolling(20).mean()
    df['Rel_Vol'] = df['Volume'] / df['Vol_Avg']
    
    # Gaps
    df['Gap'] = (df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1) * 100
    
    # Performance
    df['Return_1M'] = df['Close'].pct_change(21) * 100
    df['Return_3M'] = df['Close'].pct_change(63) * 100
    
    return df

# Process all stocks
processed = {}
for ticker, df in stock_data.items():
    processed[ticker] = add_all_indicators(df)

print(f"Processed {len(processed)} stocks with all indicators")

In [ ]:
def comprehensive_scanner(stock_data, sector_data):
    """Scan stocks using all Week 2 criteria."""
    results = []
    spy = sector_data.get('SPY')
    
    for ticker, df in stock_data.items():
        if len(df) < 200:
            continue
            
        df = add_all_indicators(df)
        latest = df.iloc[-1]
        prev = df.iloc[-2]
        
        # Trend analysis
        above_200ma = latest['Close'] > latest['SMA_200'] if pd.notna(latest['SMA_200']) else False
        above_50ma = latest['Close'] > latest['SMA_50'] if pd.notna(latest['SMA_50']) else False
        golden_cross = latest['SMA_50'] > latest['SMA_200'] if pd.notna(latest['SMA_50']) and pd.notna(latest['SMA_200']) else False
        adx_trending = latest['ADX'] > 25 if pd.notna(latest['ADX']) else False
        
        # Trend score (0-4)
        trend_score = sum([above_200ma, above_50ma, golden_cross, adx_trending])
        
        # Breakout analysis
        breakout = latest['Close'] > latest['Resistance_20'] if pd.notna(latest['Resistance_20']) else False
        breakdown = latest['Close'] < latest['Support_20'] if pd.notna(latest['Support_20']) else False
        volume_confirm = latest['Rel_Vol'] > 1.5 if pd.notna(latest['Rel_Vol']) else False
        
        # Breakout score
        breakout_score = 2 if (breakout and volume_confirm) else (1 if breakout else 0)
        
        # Relative strength vs SPY
        if spy is not None and len(spy) > 21:
            spy_ret = (spy['Close'].iloc[-1] / spy['Close'].iloc[-22] - 1) * 100
            stock_ret = latest['Return_1M'] if pd.notna(latest['Return_1M']) else 0
            rs_vs_spy = stock_ret - spy_ret
        else:
            rs_vs_spy = 0
        
        # Gap analysis (recent)
        recent_gap = abs(latest['Gap']) > 2 if pd.notna(latest['Gap']) else False
        
        # Composite score
        composite = trend_score * 2 + breakout_score + (1 if rs_vs_spy > 0 else 0)
        
        # Setup type
        if breakout and volume_confirm and trend_score >= 3:
            setup = 'STRONG BREAKOUT'
        elif trend_score >= 3 and above_50ma:
            setup = 'TREND FOLLOW'
        elif breakout:
            setup = 'WATCHLIST'
        elif trend_score <= 1:
            setup = 'AVOID'
        else:
            setup = 'NEUTRAL'
        
        results.append({
            'Ticker': ticker,
            'Price': latest['Close'],
            '1M Ret %': latest['Return_1M'] if pd.notna(latest['Return_1M']) else 0,
            'Trend': trend_score,
            'Breakout': breakout_score,
            'RS vs SPY': rs_vs_spy,
            'ADX': latest['ADX'] if pd.notna(latest['ADX']) else 0,
            'Rel Vol': latest['Rel_Vol'] if pd.notna(latest['Rel_Vol']) else 0,
            'Score': composite,
            'Setup': setup
        })
    
    return pd.DataFrame(results).sort_values('Score', ascending=False)

# Run scanner
scan_results = comprehensive_scanner(processed, sector_data)

print(f"\n{'='*80}")
print("COMPREHENSIVE POSITION TRADING SCANNER")
print(f"{'='*80}\n")
print(scan_results.round(2).to_string(index=False))

In [ ]:
def analyze_sectors_relative_strength(sector_data, sectors_dict):
    """Analyze sector relative strength."""
    results = []
    spy = sector_data.get('SPY')
    
    if spy is None:
        return pd.DataFrame()
    
    for ticker, name in sectors_dict.items():
        df = sector_data.get(ticker)
        if df is None or len(df) < 63:
            continue
        
        # Calculate relative strength
        rs = df['Close'] / spy['Close'].reindex(df.index)
        rs_ma = rs.rolling(20).mean()
        
        # Performance
        perf_1m = (df['Close'].iloc[-1] / df['Close'].iloc[-21] - 1) * 100
        perf_3m = (df['Close'].iloc[-1] / df['Close'].iloc[-63] - 1) * 100
        
        # RS trend
        rs_rising = rs.iloc[-1] > rs_ma.iloc[-1] if pd.notna(rs_ma.iloc[-1]) else False
        
        results.append({
            'Sector': name,
            'Ticker': ticker,
            '1M %': perf_1m,
            '3M %': perf_3m,
            'RS Trend': 'UP' if rs_rising else 'DOWN',
            'Rating': 'FAVOR' if rs_rising and perf_1m > 0 else ('AVOID' if not rs_rising and perf_1m < 0 else 'NEUTRAL')
        })
    
    return pd.DataFrame(results).sort_values('1M %', ascending=False)

sector_analysis = analyze_sectors_relative_strength(sector_data, sectors)

print(f"\n{'='*60}")
print("SECTOR RELATIVE STRENGTH ANALYSIS")
print(f"{'='*60}\n")
print(sector_analysis.round(2).to_string(index=False))

In [ ]:
def generate_trade_plan(scan_results, sector_analysis, capital=10000):
    """Generate complete trade plan from scan results."""
    print(f"\n{'='*70}")
    print("COMPLETE POSITION TRADING PLAN")
    print(f"{'='*70}")
    print(f"\nCapital: ${capital:,}")
    print(f"Max Position Size: ${capital * 0.2:,.0f} (20%)")
    print(f"Max Risk Per Trade: ${capital * 0.01:,.0f} (1%)")
    
    # Sector recommendations
    print(f"\n{'-'*70}")
    print("SECTOR ALLOCATION:")
    print(f"{'-'*70}")
    
    favored = sector_analysis[sector_analysis['Rating'] == 'FAVOR']
    avoided = sector_analysis[sector_analysis['Rating'] == 'AVOID']
    
    if len(favored) > 0:
        print("\nFAVORED SECTORS (overweight):")
        for _, row in favored.iterrows():
            print(f"  - {row['Sector']} ({row['Ticker']}): {row['1M %']:+.1f}% | RS: {row['RS Trend']}")
    
    if len(avoided) > 0:
        print("\nAVOID SECTORS (underweight):")
        for _, row in avoided.iterrows():
            print(f"  - {row['Sector']} ({row['Ticker']}): {row['1M %']:+.1f}% | RS: {row['RS Trend']}")
    
    # Stock recommendations
    print(f"\n{'-'*70}")
    print("TRADE CANDIDATES:")
    print(f"{'-'*70}")
    
    strong_setups = scan_results[scan_results['Setup'].isin(['STRONG BREAKOUT', 'TREND FOLLOW'])]
    
    if len(strong_setups) > 0:
        print("\nHIGH CONVICTION SETUPS:")
        for _, row in strong_setups.head(3).iterrows():
            # Calculate position size
            risk_amount = capital * 0.01  # 1% risk
            stop_distance = row['Price'] * 0.05  # 5% stop
            shares = int(risk_amount / stop_distance)
            position_value = shares * row['Price']
            
            print(f"\n  {row['Ticker']} - {row['Setup']}")
            print(f"    Current Price: ${row['Price']:.2f}")
            print(f"    Trend Score: {row['Trend']}/4 | ADX: {row['ADX']:.1f}")
            print(f"    RS vs SPY: {row['RS vs SPY']:+.1f}%")
            print(f"    Suggested Position: {shares} shares (${position_value:,.0f})")
            print(f"    Stop Loss: ${row['Price'] * 0.95:.2f} (5% below)")
            print(f"    Target: ${row['Price'] * 1.15:.2f} (15% above, 3:1 R:R)")
    
    # Watchlist
    watchlist = scan_results[scan_results['Setup'] == 'WATCHLIST']
    if len(watchlist) > 0:
        print(f"\nWATCHLIST (wait for confirmation):")
        for _, row in watchlist.head(3).iterrows():
            print(f"  - {row['Ticker']}: ${row['Price']:.2f} | Score: {row['Score']}")
    
    # Avoid
    avoid = scan_results[scan_results['Setup'] == 'AVOID']
    if len(avoid) > 0:
        print(f"\nAVOID (weak trends):")
        for _, row in avoid.iterrows():
            print(f"  - {row['Ticker']}: Trend {row['Trend']}/4 | RS: {row['RS vs SPY']:+.1f}%")
    
    print(f"\n{'='*70}")
    print("RISK REMINDERS:")
    print(f"{'='*70}")
    print("[ ] Check earnings dates before entry (avoid if within 2 weeks)")
    print("[ ] Confirm volume on entry day")
    print("[ ] Set stops immediately after entry")
    print("[ ] Review sector strength weekly")
    print("[ ] Max 5 positions at once")

generate_trade_plan(scan_results, sector_analysis)

In [ ]:
def plot_best_setup(stock_data, ticker):
    """Plot comprehensive chart for best setup."""
    if ticker not in stock_data:
        print(f"{ticker} not found")
        return
    
    df = add_all_indicators(stock_data[ticker]).tail(90)
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 12), height_ratios=[3, 1, 1])
    
    dates = range(len(df))
    
    # Price and MAs
    ax1 = axes[0]
    ax1.fill_between(dates, df['Low'], df['High'], alpha=0.2)
    ax1.plot(dates, df['Close'], 'b-', lw=1.5, label='Close')
    ax1.plot(dates, df['SMA_20'], 'orange', lw=1, label='SMA 20')
    ax1.plot(dates, df['SMA_50'], 'purple', lw=1, label='SMA 50')
    ax1.plot(dates, df['Resistance_20'], 'red', linestyle='--', lw=1, label='Resistance')
    ax1.plot(dates, df['Support_20'], 'green', linestyle='--', lw=1, label='Support')
    
    # Mark breakouts
    for i, idx in enumerate(df.index):
        row = df.loc[idx]
        if row['Close'] > row['Resistance_20'] and row['Rel_Vol'] > 1.5:
            ax1.scatter(i, row['High'], color='green', s=100, marker='^', zorder=5)
    
    ax1.set_title(f'{ticker} - Week 2 Complete Analysis')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # ADX
    ax2 = axes[1]
    ax2.plot(dates, df['ADX'], 'purple', lw=1.5)
    ax2.axhline(y=25, color='red', linestyle='--', label='Trend Threshold')
    ax2.fill_between(dates, 0, df['ADX'], where=df['ADX'] > 25, alpha=0.3, color='green')
    ax2.set_ylabel('ADX')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Volume
    ax3 = axes[2]
    colors = ['green' if df['Rel_Vol'].iloc[i] > 1.5 else 'gray' for i in range(len(df))]
    ax3.bar(dates, df['Rel_Vol'], color=colors, alpha=0.7)
    ax3.axhline(y=1.5, color='red', linestyle='--', label='Volume Threshold')
    ax3.set_ylabel('Relative Volume')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot top setup
if len(scan_results) > 0:
    best = scan_results.iloc[0]['Ticker']
    plot_best_setup(processed, best)

---

## Quiz

In [ ]:
quiz = [
    {"q": "What ADX value indicates a trending market?", "a": "B", 
     "opts": ["A) ADX > 10", "B) ADX > 25", "C) ADX < 25", "D) ADX = 50"]},
    {"q": "What volume confirms a valid breakout?", "a": "C",
     "opts": ["A) Below average", "B) At average", 
              "C) 1.5x+ above average", "D) Declining volume"]},
    {"q": "Rising relative strength vs SPY indicates?", "a": "A",
     "opts": ["A) Stock/sector outperforming market", "B) Stock/sector underperforming", 
              "C) Market is trending down", "D) Time to sell"]},
    {"q": "When should you exit before earnings?", "a": "D",
     "opts": ["A) Never, always hold through", "B) Only if losing", 
              "C) Only if winning", "D) If you're a short-term trader"]},
    {"q": "What's the typical win rate for trend following?", "a": "B",
     "opts": ["A) 60-70%", "B) 35-45%", 
              "C) 80-90%", "D) 20-25%"]},
    {"q": "Which sectors lead in early expansion?", "a": "A",
     "opts": ["A) Tech, Financials, Consumer Disc.", "B) Utilities, Healthcare", 
              "C) Energy, Materials", "D) Consumer Staples"]},
    {"q": "Where should stops be placed for breakout longs?", "a": "B",
     "opts": ["A) Above the breakout level", "B) Below the breakout level", 
              "C) At entry price", "D) No stop needed"]},
    {"q": "What is the Golden Cross signal?", "a": "C",
     "opts": ["A) Price crosses above 200 MA", "B) 20 MA crosses above 50 MA", 
              "C) 50 MA crosses above 200 MA", "D) ADX crosses above 25"]},
    {"q": "Post-earnings 'gap and go' means?", "a": "A",
     "opts": ["A) Gap with follow-through in gap direction", "B) Gap that fills", 
              "C) Gap that reverses", "D) No gap at all"]},
    {"q": "Ideal R:R for position trades?", "a": "C",
     "opts": ["A) 1:1", "B) 1:2", 
              "C) 2:1 or 3:1", "D) 5:1 minimum"]}
]

def run_quiz():
    print("WEEK 2 COMPREHENSIVE QUIZ")
    print("="*50)
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-B, 2-C, 3-A, 4-D, 5-B, 6-A, 7-B, 8-C, 9-A, 10-C")
    
run_quiz()

---

## Summary

### Week 2 Key Takeaways

1. **Trend Following**: Low win rate, high profit factor; use ADX > 25 to confirm trends
2. **Breakouts**: Volume > 1.5x confirms; stop below breakout level
3. **Sector Rotation**: Follow relative strength; rotate into leaders
4. **Earnings**: Capture run-up or trade gaps; reduce size before reports

### Integration Tips

- Combine sector strength with stock selection
- Use trend confirmation before breakout trades
- Always check earnings calendar
- Position size based on stop distance

**Next Week**: Week 3 - Value & Growth Investing